In [1]:
from jellyfish._jellyfish import damerau_levenshtein_distance
import distance
from sklearn import metrics

In [2]:
from keras.models import load_model
from theano.ifelse import ifelse #added this

Using Theano backend.


In [3]:
import pickle
import pandas as pd
import numpy as np
from utils_test import *

In [4]:
test = pd.read_csv('../input/test.csv')
test = transformDf(test)

In [5]:
with open('../input/parameters.pkl', "rb") as f:
    maxlen = pickle.load(f)
    num_features = pickle.load(f)
    chartoindice = pickle.load(f)
    targetchartoindice = pickle.load(f)
    divisor = pickle.load(f)
    divisor2 = pickle.load(f)

In [6]:
with open('../input/preprocessed_data.pkl', "rb") as f:
    X = pickle.load(f)
    y_a = pickle.load(f)
    y_t = pickle.load(f)
    X_test = pickle.load(f)
    y_a_test = pickle.load(f)
    y_t_test = pickle.load(f)

# Utils

In [7]:
model = load_model('output_files/models/model_31-0.95.h5')

/Users/hoangnguyen/miniconda3/envs/pydata2/lib/python2.7/site-packages/theano/tensor/basic.py:5130: UserWarning: flatten outdim parameter is deprecated, use ndim instead.
  "flatten outdim parameter is deprecated, use ndim instead.")


In [8]:
test_groupByCase = test.groupby(['CaseID'])

In [9]:
test_len = findLongestLength(test_groupByCase)
test_len - 5

8

In [10]:
sentences, sentences_t, sentences_t2, sentences_t3, sentences_t4 = getFeature(test_groupByCase)

In [11]:
next_chars, next_chars_t, next_chars_t2, next_chars_t3, next_chars_t4 = getOutput(test_groupByCase)

# Suffix

## Predict

In [12]:
suffix = 2 #Change in the interval [2, 7]

In [13]:
sentences_2 = []
sentences_t_2 = []
sentences_t2_2 = []
sentences_t3_2 = []
sentences_t4_2 = []
next_chars_2 = []
next_chars_t_2 = []

for i in range(len(sentences)):
    if len(sentences[i]) == suffix:
        sentences_2.append(sentences[i])
        sentences_t_2.append(sentences_t[i])
        sentences_t2_2.append(sentences_t2[i])
        sentences_t3_2.append(sentences_t3[i])
        sentences_t4_2.append(sentences_t4[i])
        next_chars_2.append(next_chars[i])
        next_chars_t_2.append(next_chars_t[i])

In [14]:
len(sentences_2), len(next_chars_2)

(1268, 1268)

In [15]:
X_test = vectorizeInput(sentences_2, sentences_t_2, sentences_t2_2, sentences_t3_2, sentences_t4_2, 
                        maxlen, num_features, chartoindice, 
                        divisor, divisor2, divisor3=86400, divisor4=7)

In [16]:
pred = model.predict(X_test, verbose=0)
pred_a = pred[0]
pred_t = pred[1]

In [17]:
pred[0].shape, pred[1].shape

((1268, 10), (1268, 1))

## Evaluate

### Activity

In [18]:
#for log-loss
y_a_test = one_hot_encode(next_chars_2, targetchartoindice)

In [19]:
metrics.log_loss(y_a_test, pred_a)

0.66368157546438444

In [20]:
#for accuracy
gt_a_label = next_chars_2
pred_a_label = getLabel(pred_a, targetchartoindice)

In [21]:
metrics.accuracy_score(gt_a_label, pred_a_label)

0.77917981072555209

In [22]:
get_top3_accuracy(pred_a, gt_a_label, targetchartoindice)

0.9873817034700315

In [23]:
# DL distance
distance.nlevenshtein(gt_a_label, pred_a_label)

0.2200315457413249

In [24]:
damerau_levenshtein_distance(gt_a_label, pred_a_label)

279

In [25]:
distance.jaccard(gt_a_label, pred_a_label)

0.5

### Time

In [26]:
y_t_test = normalize(next_chars_t_2, divisor)

In [27]:
gt_t = next_chars_t_2

In [28]:
metrics.mean_squared_error(gt_t, pred_t)

367851182167.78821

In [29]:
metrics.mean_absolute_error(gt_t, pred_t)

282811.59823356668

In [30]:
metrics.median_absolute_error(gt_t, pred_t)

10778.880826279521

In [31]:
# Convert to day

In [32]:
metrics.mean_absolute_error(gt_t, pred_t)/86400

3.2732823869625771

In [33]:
metrics.median_absolute_error(gt_t, pred_t)/86400

0.12475556511897594